In [1]:
from utils_2 import *
from model_3 import *
from dataset_2 import *
from train_2 import *
from predict_2 import *

In [2]:

# Load You dataset
x = np.random.rand(1,125,125,3)
y = 1.0

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [4]:
dataset = CustomDataset(x, y, transform,mode = 'test')

In [5]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [7]:
# Since Kfold training has been performed we have ensemble of kfold models
k_folds = 5

outputs = []
models = []
predicts = []
for fold in range(k_folds):
    DEVICE = torch.device("cuda")
    model = VGG_12(BasicBlock, 3, 16, CBAM)
    model = nn.DataParallel(model)
    model.load_state_dict(torch.load(f'./vggWeights/best_model_{fold}.pth'))
    model.eval()
    model = model.to('cuda')
    predict = Predict(model)
    predicts.append(predict)



for batch in dataloader:
    op = []
    img = batch['image'].to('cuda').float()
    for fold in range(k_folds):
        op.append(predicts[fold].predict(img).cpu().detach().numpy())
    op = np.array(op)
    outputs.append(op.mean())

# outputs.append(np.array(op))